# Data Science Capstone Project


## Purpose
The purpose of this project is to explore the venues of Toronto and Waterloo and understand the differences between a big city and a smaller city.

## Background
Predicting where to open a business is a very hard problem. The number of variables that go into solving this kind of problem is a lot. It is almost impossible to predict whether a certain kind of business might succeed in one neighbourhood vs the other, one city vs the other, Evironment, culture, people, religion, time, location, asthetic appeal they all intermingle with each other to predict whether a certain place might be a good place for a business. Also what kind of business also makes a big difference. I am no expert and neither do I claim to be I just experiment and analyze and predict. So I just go into this problem blindly and see what comes out of the end. In this project I look into one variable which is the venue data on othere people's busineses and I cluster them and see which cluster is good for which kind of business.


## Methodology
I. Data acquisiton and cleaning

II. Data Mining and Cleaning

III. Exploratory Data Analysis

IV. Predictive Modeling

V. Conclusions

## WIP - improvements

Notable TODOs:
- todo 1: Deciding what kind of data I need and get relavant data.
- todo 2: If correlation and conclusions can be drawn.
- todo 3: Tasty visualizations.
- Choropleth maps of neighbourhoods comparing number of venues.



# Setup

## Library import
We import all the required Python libraries

In [ ]:
# Imported all the neccesary libraries 
from bs4 import BeautifulSoup # For Web Scraping

import csv
import pandas as pd #Data Analysis
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)
import numpy as np

import geocoder #get location data
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests
import json #To handle json files
from pandas.io.json import json_normalize

from sklearn.cluster import KMeans # To run k-means algorithm

import matplotlib.cm as cm
import matplotlib.colors as colors # matplotlib stuff
import matplotlib.pyplot as plt
#%matplotlib inline # backend for rendering plots wit

import folium # To visualize on map
from folium.map import *
from folium import plugins
from folium.plugins import MeasureControl
from folium.plugins import FloatImage

import json

print('Libraries Imported')


# Data import
We retrieve all the required data for the analysis. I use the arcgis API to get the required Waterloo-Kitchener Neighbourhood Data. I am aware that these Neighbourhoods might not be accurate in their location but these are just rough placeholder names to segment the area.

In [ ]:
url = 'https://services.arcgis.com/ZpeBVw5o1kjit7LT/arcgis/rest/services/NeighbourhoodAssociations/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'
data = requests.get(url).json()  


In [ ]:
with open('data.geojson', 'w') as json_file:
    json.dump(data, json_file)

In [ ]:
address = 'Waterloo, Ontario, Canada'

geolocator = Nominatim(user_agent="waterloo_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Waterloo City are {}, {}.'.format(latitude, longitude))

In [ ]:
element = []

for item in data['features']:
    i = round(len(item['geometry']['rings'][0])/2)+round(len(item['geometry']['rings'][0])/3)
    element.append([item['attributes']['NAME'],item['attributes']['TYPE'],item['geometry']['rings'][0][i][1],item['geometry']['rings'][0][i][0]])
    
element[:5]

In [ ]:
df = pd.DataFrame(element,columns = ['Neighbourhood', 'Neighbourhood Type','lat','long'])

In [ ]:
df.shape

In [ ]:
# # Displaying toronto areas by postal Code
# for lat,lng,neighbourhood in zip(df['lat'],df['long'],df['Neighbourhood Type']):
#     label = f'{neighbourhood}'
#     label = folium.Popup(label, parse_html=True)
#     folium.CircleMarker(
#         [lat,lng],
#         radius = 5,
#         popup=label,
#         color = 'green',
#         fill = True,
#         parse_html=False, 
#     ).add_to(waterloo_map)
    
# # waterloo_map.add_child(MeasureControl())
# FloatImage(url, bottom=5, left=85).add_to(waterloo_map)
# waterloo_map.add_child(MeasureControl())
# waterloo_map.save('index.html')

# waterloo_map

Getting Kitchener Data

In [ ]:
url = 'https://services1.arcgis.com/qAo1OsXi67t7XgmS/arcgis/rest/services/Neighbourhood_Association/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'

In [ ]:
data2 = requests.get(url).json() 

In [ ]:
data2['features'][0]

In [ ]:
element = []

for item in data2['features']:
    i = round(len(item['geometry']['rings'][0])/5)
    element.append([item['attributes']['MAPLABEL1'],item['attributes']['NEIGHBOURHOOD_ASSOCIATION'],item['geometry']['rings'][0][i][1],item['geometry']['rings'][0][i][0]])
    
element[:5]

In [ ]:
df2 = pd.DataFrame(element,columns = ['Neighbourhood', 'Neighbourhood Type','lat','long'])

In [ ]:
df2.head()

In [ ]:
df_big = pd.concat([df,df2],axis = 0,sort = False).reset_index()
df_big.to_csv(r'C:\Users\Abhik\Downloads\df_big.csv',index = False)

In [ ]:
df_big.head()


In [ ]:
df_big.shape

We have our desired Dataframe above.

In [ ]:
address = 'Waterloo, Ontario, Canada'

geolocator = Nominatim(user_agent="waterloo_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Waterloo City are {}, {}.'.format(latitude, longitude))

In [ ]:
KW_map = folium.Map(location=[latitude, longitude], zoom_start=12)

KW_map

In [ ]:
for lat,lng,neighbourhood in zip(df_big['lat'],df_big['long'],df_big['Neighbourhood Type']):
    label = f'{neighbourhood}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        popup=label,
        color = 'green',
        fill = True,
        parse_html=False, 
    ).add_to(KW_map)
    
# waterloo_map.add_child(MeasureControl())
FloatImage(url, bottom=5, left=85).add_to(KW_map)
KW_map.add_child(MeasureControl())

KW_map

In [ ]:
#KW2_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# KW2_map
# for item in data['features']:
#     coords = item['geometry']['rings'][0]
#     foo = pd.DataFrame(coords,columns=['long','lat'])
#     for lat,lng in zip(foo['lat'],foo['long']):
#     #     lat = foo['lat'][0]
#     #     lng = foo['long'][0]
#         folium.CircleMarker(
#             [lat,lng],
#             radius = 0.5,
#             color = 'green',
#             fill = True,
#             parse_html=False, 
#         ).add_to(KW2_map)

# KW2_map
# foo['lat'][0]

In [ ]:
lat = df_big['lat'][0]
long = df_big['long'][0]
radius = 500

CLIENT_ID = 'JR0WCEE3K2VYBHOHIRHCAPOL3BXLGM2WHQNPCXXJ0GB20HPP' #Foursquare ID
CLIENT_SECRET = '251PHKQSNFRZEAMTRWB3YKHYRHT50WRVAAW35AMS3CHXDFE2' # Foursquare Secret
VERSION = '20200516' # Foursquare API version
LIMIT = 500

url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{long}&radius={radius}&limit={LIMIT}'
json = requests.get(url).json()
json

In [ ]:
for item in json['response']['groups'][0]['items']:
    print(item['venue']['location']['lat'])

In [ ]:
names = df_big['Neighbourhood']
latitudes = df_big['lat']
longtitudes = df_big['long']
radius = 500

CLIENT_ID = 'JR0WCEE3K2VYBHOHIRHCAPOL3BXLGM2WHQNPCXXJ0GB20HPP' #Foursquare ID
CLIENT_SECRET = '251PHKQSNFRZEAMTRWB3YKHYRHT50WRVAAW35AMS3CHXDFE2' # Foursquare Secret
VERSION = '20200528' # Foursquare API version
LIMIT = 500

list = []
bitch = []


for name,lat,long in zip(names,latitudes,longtitudes):
    url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{long}&radius={radius}&limit={LIMIT}'
    json = requests.get(url).json()
    venues = json['response']['groups'][0]['items']
    try:
        list.append([(
                    name,
                    lat,
                    long,
                    v['venue']['name'], 
                    v['venue']['location']['lat'],
                    v['venue']['categories'][0]['name'],
                    v['venue']['location']['lng']) for v in venues]
                   )
    except:
        continue


In [ ]:
bitch = []
for name,lat,long in zip(names,latitudes,longtitudes):
    url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{long}&radius={radius}&limit={LIMIT}'
    json = requests.get(url).json()
    try:
        venues = json['response']['groups'][0]['items']
        if len(venues) == 0:
            bitch.append(name)
    except:
        continue
bitch

In [ ]:
list

In [ ]:
# dataframe = pd.DataFrame([item for venue_list in list for item in venue_list],
#                          columns = ['Postal Code','Borough','lat','long','venue','venue lat','venue long','venue type'])

In [ ]:
venue_stuff = pd.DataFrame([item for venue_list in list for item in venue_list],columns = ['Neighbourhood','lat','long','venue','venue lat','venue type','venue long'])

In [ ]:
venue_stuff.head()

In [ ]:
venue_stuff.to_csv(r'C:\Users\Abhik\Downloads\venue_waterloo_data.csv',index = False)

data saved as csv as venue_waterloo_data.csv

In [ ]:
x = pd.DataFrame(venue_stuff.groupby(['venue type','Neighbourhood']).count()['lat']).reset_index()

In [ ]:
x['venue type'].value_counts()

In [ ]:
# One Hot Encoding
waterloo_onehot = pd.get_dummies(venue_stuff[['venue type']],prefix ='',prefix_sep = '')
waterloo_onehot['Neighbourhood'] = venue_stuff['Neighbourhood']

fixed_columns = [waterloo_onehot.columns[-1]] + waterloo_onehot.columns[:-1].to_list()

waterloo_onehot = waterloo_onehot[fixed_columns]

len(waterloo_onehot['Neighbourhood'].value_counts())

In [ ]:
waterloo_grouped = waterloo_onehot.groupby('Neighbourhood').mean().reset_index()

In [ ]:
waterloo_grouped.shape

In [ ]:
waterloo_clustered = waterloo_grouped.drop('Neighbourhood',1)

In [ ]:
sns.heatmap(waterloo_clustered,cmap="YlGnBu")

In [ ]:
# run k-means clustering
kclusters = 15
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(waterloo_clustered)

# check cluster labels generated for each row in the dataframe
len(kmeans.labels_)

In [ ]:
df_big.shape

In [ ]:
for item in bitch:    
    df_big = df_big[df_big['Neighbourhood'] != item]

df_big.to_csv(r'C:\Users\Abhik\Downloads\df_big.csv',index = False)

In [ ]:
df_big.head()

In [ ]:
#df_big = df_big.drop('Cluster Labels',1)
df_big.insert(0, 'Cluster Labels', kmeans.labels_)

In [ ]:
df_big.head()

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_big['lat'], df_big['long'], df_big['Neighbourhood'], df_big['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       

map_clusters

In [ ]:
inertia=[]
N = 55
K = []
for k in range(2,N) :
    K.append(k)
    kmeans = KMeans(n_clusters=k, random_state=0).fit(waterloo_clustered)
    inertia.append(kmeans.inertia_)
plt.scatter(x = inertia,y =K)
plt.xlabel('inertia values')
plt.ylabel('k values')
plt.title('elbow plot')

In [ ]:
from IPython.display import display

for i in range(kclusters):
    print(f"TABLE CLUSTER {i}")
    display(df_big[df_big['Cluster Labels'] == i])

In [ ]:
for i in range(kclusters):
    print(f'cluster {i}')
    venue_cluster = df_big[df_big['Cluster Labels'] == i]
    for item in venue_cluster['Neighbourhood']:        
        display(venue_stuff[venue_stuff['Neighbourhood'] == item].drop('Neighbourhood',1))

In [ ]:
venues = json['response']['groups'][0]['items']
gay = []
gay.append([(v['venue']['name']) for v in venues])

In [ ]:
json

In [ ]:
venue_id = []
for name,lat,long in zip(names,latitudes,longtitudes):
    url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{long}&radius={radius}&limit={LIMIT}'
    json = requests.get(url).json()
    try:
        venues = json['response']['groups'][0]['items']
        venue_id.append([(
                    name,
                    lat,
                    long,
                    v['venue']['name'], 
                    v['venue']['id']) for v in venues]
                   )
    except:
        continue


In [ ]:
venue_id

In [ ]:
venue_id_df = pd.DataFrame([item for venue_list in venue_id for item in venue_list],columns = ['Neighbourhood','lat','long','venue name','id'])

In [ ]:
display(venue_id_df.head())
display(venue_id_df.shape)

In [ ]:
json['response']['groups'][0]['items'][0]['venue']['id']

In [ ]:
foo = []
venues = json['response']['groups'][0]['items']
foo.append([(v['venue']['id']) for v in venues])


In [ ]:
venue_id = '5251b49a11d2c68b0e3b8a65'
url = f'https://api.foursquare.com/v2/venues/{venue_id}?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}' 
result = requests.get(url).json()

In [ ]:
result

In [ ]:
ratings = []
venue_namez = venue_id_df['venue name'] 
venue_id = venue_id_df['id']
for name,id in zip(venue_namez,venue_id):
    url = f'https://api.foursquare.com/v2/venues/{id}?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}'
    result = requests.get(url).json()
    try:    
        ratings.append([result['response']['venue']['rating'],name])
    except:
        continue


In [ ]:
pd.DataFrame(ratings)

In [ ]:
KW_map = folium.Map(location=[latitude, longitude],tiles="Stamen Toner", zoom_start=12)


for lat,lng,name in zip(venue_stuff['venue lat'],venue_stuff['venue long'],venue_stuff['venue']):
    label = f'{name}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius = 2,
        popup=label,
        color = 'red',
        fill = True,
        parse_html=False, 
    ).add_to(KW_map)
    


KW_map

In [ ]:
toronto_venue_data = pd.read_csv('venue_toronto_data.csv')

In [ ]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="yyz_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

In [ ]:
map_YYZ = folium.Map(location=[latitude, longitude],tiles = 'Stamen Toner', zoom_start=11) #Displaying Toronto Map

for lat,lng,name in zip(toronto_venue_data['venue lat'],toronto_venue_data['venue long'],toronto_venue_data['venue']):
    label = f'{name}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius = 2,
        popup=label,
        color = 'red',
        fill = True,
        parse_html=False, 
    ).add_to(map_YYZ)
    
map_YYZ

In [ ]:
toronto_clusters = pd.read_csv('toronto_clusters.csv')

In [ ]:
toronto_clusters.head()

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

kclusters = 30
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_clusters['lat'], toronto_clusters['long'], toronto_clusters['Borough'], toronto_clusters['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       

map_clusters

In [ ]:
cluster_i_neighbourhood = toronto_clusters[toronto_clusters['Cluster Labels'] == 0].reset_index()['Borough'].to_list()
cluster_i_neighbourhood

In [ ]:
cluster_i_neighbourhood[1]

In [ ]:
len(toronto_clusters['Cluster Labels'])

In [ ]:
len(toronto_venue_data[toronto_venue_data['Borough']==cluster_i_neighbourhood[1]])

In [ ]:
toronto_clusters[toronto_clusters['Cluster Labels']==1]


In [ ]:
for i in toronto_venue_data.iterrows():
    print(i)

In [ ]:
dataframe = []
for j in range(len(toronto_clusters['Cluster Labels'])):
    cluster_i_neighbourhood = toronto_clusters[toronto_clusters['Cluster Labels'] == j].reset_index()['Borough'].to_list()
    for i in range(len(cluster_i_neighbourhood)):
        cluster_list = np.full(shape=len(toronto_venue_data[toronto_venue_data['Borough']==cluster_i_neighbourhood[i]]),
                           fill_value=i).tolist()
        temp = toronto_venue_data[toronto_venue_data['Borough']==cluster_i_neighbourhood[i]]
        temp.insert(0,'Cluster Labels',cluster_list)
        dataframe.append(temp)

In [ ]:
len(dataframe[0])

In [ ]:
dataframe[5][0]

In [ ]:
venue_cluster = pd.concat(dataframe[i] for i in range(len(toronto_clusters['Cluster Labels']))).reset_index(drop=True)
venue_cluster.drop_duplicates().shape

In [ ]:
toronto_venue_data.shape

In [ ]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="yyz_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_clusters = folium.Map(location=[latitude, longitude],tiles='Stamen Toner', zoom_start=11)

kclusters = 30
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(len(toronto_clusters['Cluster Labels']))]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(venue_cluster['venue lat'], venue_cluster['venue long'], venue_cluster['Borough'], venue_cluster['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=1,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       

map_clusters



In [ ]:
cluster_6 = venue_cluster[venue_cluster['Cluster Labels']==0]


In [ ]:
cluster_6.head()

In [ ]:
display(pd.DataFrame(cluster_6['venue type'].value_counts()).reset_index().head(15))
display(pd.DataFrame(cluster_6['Borough'].value_counts()).head(15))

In [ ]:
dataframe = []
for j in range(len(df_big['Cluster Labels'])):
    cluster_i_neighbourhood = df_big[df_big['Cluster Labels'] == j].reset_index()['Neighbourhood'].to_list()
    for i in range(len(cluster_i_neighbourhood)):
        cluster_list = np.full(shape=len(venue_stuff[venue_stuff['Neighbourhood']==cluster_i_neighbourhood[i]]),
                           fill_value=i).tolist()
        temp = venue_stuff[venue_stuff['Neighbourhood']==cluster_i_neighbourhood[i]]
        temp.insert(0,'Cluster Labels',cluster_list)
        dataframe.append(temp)

In [ ]:
waterloo_new_venue = pd.concat(dataframe).reset_index(drop = True)

## yote

In [ ]:
address = 'Waterloo, Ontario, Canada'

geolocator = Nominatim(user_agent="waterloo_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


map_clusters = folium.Map(location=[latitude, longitude],tiles = 'Stamen Toner', zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(waterloo_new_venue['venue lat'], waterloo_new_venue['venue long'], waterloo_new_venue['Neighbourhood'], waterloo_new_venue['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=2,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       

map_clusters